In [1]:
import itertools
import logging
import re
import time
import traceback
from datetime import datetime
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ChromeOptions as Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

GM_WEBPAGE = 'https://www.google.com/maps/'
MAX_WAIT = 10
MAX_RETRY = 5
MAX_SCROLLS = 40

class GoogleMapsScraper:

    def __init__(self, debug=False):
        self.debug = debug
        self.driver = self.__get_driver()
        self.logger = self.__get_logger()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, tb):
        if exc_type is not None:
            traceback.print_exception(exc_type, exc_value, tb)

        self.driver.close()
        self.driver.quit()

        return True

    def sort_by(self, url, ind):

        self.driver.get(url)
        self.__click_on_cookie_agreement()

        wait = WebDriverWait(self.driver, MAX_WAIT)

        # open dropdown menu
        clicked = False
        tries = 0
        while not clicked and tries < MAX_RETRY:
            try:
                menu_bt = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@data-value=\'Sort\']')))
                menu_bt.click()

                clicked = True
                time.sleep(3)
            except Exception as e:
                tries += 1
                self.logger.warning('Failed to click sorting button')

            # failed to open the dropdown
            if tries == MAX_RETRY:
                return -1

        #  element of the list specified according to ind
        recent_rating_bt = self.driver.find_elements(By.XPATH, '//div[@role=\'menuitemradio\']')[ind]
        recent_rating_bt.click()

        # wait to load review (ajax call)
        time.sleep(5)

        return 0

    def get_places(self, keyword_list=None):

        df_places = pd.DataFrame()
        search_point_url_list = self._gen_search_points_from_square(keyword_list=keyword_list)

        for i, search_point_url in enumerate(search_point_url_list):
            print(search_point_url)

            if (i+1) % 10 == 0:
                print(f"{i}/{len(search_point_url_list)}")
                df_places = df_places[['search_point_url', 'href', 'name', 'rating', 'num_reviews', 'close_time', 'other']]
                df_places.to_csv('output/places_wax.csv', index=False)


            try:
                self.driver.get(search_point_url)
            except NoSuchElementException:
                self.driver.quit()
                self.driver = self.__get_driver()
                self.driver.get(search_point_url)

            # scroll to load all (20) places into the page
            scrollable_div = self.driver.find_element(By.CSS_SELECTOR,
                "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div[aria-label*='Results for']")
            for i in range(10):
                self.driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)

            # Get places names and href
            time.sleep(2)
            response = BeautifulSoup(self.driver.page_source, 'html.parser')
            div_places = response.select('div[jsaction] > a[href]')

            for div_place in div_places:
                place_info = {
                    'search_point_url': search_point_url.replace('https://www.google.com/maps/search/', ''),
                    'href': div_place['href'],
                    'name': div_place['aria-label']
                }

                df_places = df_places.append(place_info, ignore_index=True)

            # TODO: implement click to handle > 20 places

        df_places = df_places[['search_point_url', 'href', 'name']]
        df_places.to_csv('output/places_wax.csv', index=False)



    def get_reviews(self, offset):

        # scroll to load reviews
        self.__scroll()

        # wait for other reviews to load (ajax)
        time.sleep(4)

        # expand review text
        self.__expand_reviews()

        # parse reviews
        response = BeautifulSoup(self.driver.page_source, 'html.parser')
        # TODO: Subject to changes
        rblock = response.find_all('div', class_='jftiEf fontBodyMedium')
        parsed_reviews = []
        for index, review in enumerate(rblock):
            if index >= offset:
                r = self.__parse(review)
                parsed_reviews.append(r)

                # logging to std out
                print(r)

        return parsed_reviews



    # need to use different url wrt reviews one to have all info
    def get_account(self, url):

        self.driver.get(url)
        self.__click_on_cookie_agreement()

        # ajax call also for this section
        time.sleep(2)

        resp = BeautifulSoup(self.driver.page_source, 'html.parser')

        place_data = self.__parse_place(resp, url)

        return place_data


    def __parse(self, review):

        item = {}

        try:
            # TODO: Subject to changes
            id_review = review['data-review-id']
        except Exception as e:
            id_review = None

        try:
            # TODO: Subject to changes
            username = review['aria-label']
        except Exception as e:
            username = None

        try:
            # TODO: Subject to changes
            review_text = self.__filter_string(review.find('span', class_='wiI7pd').text)
        except Exception as e:
            review_text = None

        try:
            # TODO: Subject to changes
            rating = float(review.find('span', class_='kvMYJc')['aria-label'].split(' ')[0])
        except Exception as e:
            rating = None

        try:
            # TODO: Subject to changes
            relative_date = review.find('span', class_='rsqaWe').text
        except Exception as e:
            relative_date = None

        try:
            n_reviews = review.find('div', class_='RfnDt').text.split(' ')[3]
        except Exception as e:
            n_reviews = 0

        try:
            user_url = review.find('button', class_='WEBjve')['data-href']
        except Exception as e:
            user_url = None

        item['id_review'] = id_review
        item['caption'] = review_text

        # depends on language, which depends on geolocation defined by Google Maps
        # custom mapping to transform into date should be implemented
        item['relative_date'] = relative_date

        # store datetime of scraping and apply further processing to calculate
        # correct date as retrieval_date - time(relative_date)
        item['retrieval_date'] = datetime.now()
        item['rating'] = rating
        item['username'] = username
        item['n_review_user'] = n_reviews
        #item['n_photo_user'] = n_photos  ## not available anymore
        item['url_user'] = user_url

        return item


    def __parse_place(self, response, url):

        place = {}

        try:
            place['name'] = response.find('h1', class_='DUwDvf fontHeadlineLarge').text.strip()
        except Exception as e:
            place['name'] = None

        try:
            place['overall_rating'] = float(response.find('div', class_='F7nice ').find('span', class_='ceNzKf')['aria-label'].split(' ')[1])
        except Exception as e:
            place['overall_rating'] = None

        try:
            place['n_reviews'] = int(response.find('div', class_='F7nice ').text.split('(')[1].replace(',', '').replace(')', ''))
        except Exception as e:
            place['n_reviews'] = 0

        try:
            place['n_photos'] = int(response.find('div', class_='YkuOqf').text.replace('.', '').replace(',','').split(' ')[0])
        except Exception as e:
            place['n_photos'] = 0

        try:
            place['category'] = response.find('button', jsaction='pane.rating.category').text.strip()
        except Exception as e:
            place['category'] = None

        try:
            place['description'] = response.find('div', class_='PYvSYb').text.strip()
        except Exception as e:
            place['description'] = None

        b_list = response.find_all('div', class_='Io6YTe fontBodyMedium')
        try:
            place['address'] = b_list[0].text
        except Exception as e:
            place['address'] = None

        try:
            place['website'] = b_list[1].text
        except Exception as e:
            place['website'] = None

        try:
            place['phone_number'] = b_list[2].text
        except Exception as e:
            place['phone_number'] = None
    
        try:
            place['plus_code'] = b_list[3].text
        except Exception as e:
            place['plus_code'] = None

        try:
            place['opening_hours'] = response.find('div', class_='t39EBf GUrTXd')['aria-label'].replace('\u202f', ' ')
        except:
            place['opening_hours'] = None

        place['url'] = url

        lat, long, z = url.split('/')[6].split(',')
        place['lat'] = lat[1:]
        place['long'] = long

        return place


    def _gen_search_points_from_square(self, keyword_list=None):
        # TODO: Generate search points from corners of square

        keyword_list = [] if keyword_list is None else keyword_list

        square_points = pd.read_csv('input/square_points.csv')

        cities = square_points['city'].unique()

        search_urls = []

        for city in cities:

            df_aux = square_points[square_points['city'] == city]
            latitudes = df_aux['latitude'].unique()
            longitudes = df_aux['longitude'].unique()
            coordinates_list = list(itertools.product(latitudes, longitudes, keyword_list))

            search_urls += [f"https://www.google.com/maps/search/{coordinates[2]}/@{str(coordinates[1])},{str(coordinates[0])},{str(15)}z"
             for coordinates in coordinates_list]

        return search_urls


    # expand review description
    def __expand_reviews(self):
        # use XPath to load complete reviews
        # TODO: Subject to changes
        buttons = self.driver.find_elements(By.CSS_SELECTOR,'button.w8nwRe.kyuRq')
        for button in buttons:
            self.driver.execute_script("arguments[0].click();", button)


    def __scroll(self):
        # TODO: Subject to changes
        scrollable_div = self.driver.find_element(By.CSS_SELECTOR,'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
        self.driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        #self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


    def __get_logger(self):
        # create logger
        logger = logging.getLogger('googlemaps-scraper')
        logger.setLevel(logging.DEBUG)

        # create console handler and set level to debug
        fh = logging.FileHandler('gm-scraper.log')
        fh.setLevel(logging.DEBUG)

        # create formatter
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

        # add formatter to ch
        fh.setFormatter(formatter)

        # add ch to logger
        logger.addHandler(fh)

        return logger


    def __get_driver(self, debug=False):
        options = Options()

        if not self.debug:
            options.add_argument("--headless")
        else:
            options.add_argument("--window-size=1366,768")

        options.add_argument("--disable-notifications")
        #options.add_argument("--lang=en-GB")
        options.add_argument("--accept-lang=en-GB")
        input_driver = webdriver.Chrome(service=Service(), options=options)

         # click on google agree button so we can continue (not needed anymore)
         # EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "I agree")]')))
        input_driver.get(GM_WEBPAGE)

        return input_driver

    # cookies agreement click
    def __click_on_cookie_agreement(self):
        try:
            agree = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "Reject all")]')))
            agree.click()

            # back to the main page
            # self.driver.switch_to_default_content()

            return True
        except:
            return False

    # util function to clean special characters
    def __filter_string(self, str):
        strOut = str.replace('\r', ' ').replace('\n', ' ').replace('\t', ' ')
        return strOut

In [2]:
from datetime import datetime, timedelta
import argparse
import csv
from termcolor import colored
import time
import sys

# Sorting index mapping
ind = {'most_relevant': 0, 'newest': 1, 'highest_rating': 2, 'lowest_rating': 3}
HEADER = ['id_review', 'caption', 'relative_date', 'retrieval_date', 'rating', 'username', 'n_review_user', 'n_photo_user', 'url_user']
HEADER_W_SOURCE = ['id_review', 'caption', 'relative_date', 'retrieval_date', 'rating', 'username', 'n_review_user', 'n_photo_user', 'url_user', 'url_source']


def csv_writer(source_field, ind_sort_by, path='D:\\MTech\\4-SEM\\ProjectCode\\'):
    outfile= ind_sort_by + '_gm_reviews.csv'
    targetfile = open(path + outfile, mode='w', encoding='utf-8', newline='\n')
    writer = csv.writer(targetfile, quoting=csv.QUOTE_MINIMAL)

    if source_field:
        h = HEADER_W_SOURCE
    else:
        h = HEADER
    writer.writerow(h)

    return writer


if __name__ == '__main__':
    # Filter out unwanted arguments
    argv = sys.argv
    args_to_parse = [arg for arg in argv if arg.startswith('--')]

    parser = argparse.ArgumentParser(description='Google Maps reviews scraper.')
    parser.add_argument('--N', type=int, default=1000, help='Number of reviews to scrape')
    parser.add_argument('--i', type=str, default='D:\\MTech\\4-SEM\\ProjectCode\\URLS.txt', help='Target URLs file')
    parser.add_argument('--sort_by', type=str, default='newest', help='Sorting order: most_relevant, newest, highest_rating, or lowest_rating')
    parser.add_argument('--place', dest='place', action='store_true', help='Scrape place metadata')
    parser.add_argument('--debug', dest='debug', action='store_true', help='Run scraper using browser graphical interface')
    parser.add_argument('--source', dest='source', action='store_true', help='Add source URL to CSV file (for multiple URLs in a single file)')
    parser.set_defaults(place=False, debug=False, source=False)

    args = parser.parse_args(args_to_parse)
    
    # store reviews in CSV file
    writer = csv_writer(args.source, args.sort_by)
    
    with GoogleMapsScraper(debug=args.debug) as scraper:
        with open(args.i, 'r') as urls_file:
            for url in urls_file:
                if args.place:
                    print(scraper.get_account(url))
                else:
                    error = scraper.sort_by(url, ind[args.sort_by])

                    if error == 0:

                        n = 0

                        #if ind[args.sort_by] == 0:
                        #    scraper.more_reviews()

                        while n < args.N:

                            # logging to std out
                            print(colored('[Review ' + str(n) + ']', 'cyan'))

                            reviews = scraper.get_reviews(n)
                            if len(reviews) == 0:
                                break

                            for r in reviews:
                                row_data = list(r.values())
                                if args.source:
                                    row_data.append(url[:-1])

                                writer.writerow(row_data)

                            n += len(reviews)

    

[Review 0]
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSURidjhyRUFnEAE', 'caption': "Great service and wonderful spread of buffet.  The service was fast and they accommodated our team of six at a comfortable table.. The starters kept coming until we couldn't have no more.. a decent spread of main course dishes then followed post which we devoured on the deserts.. The kulfi faluda and those tiny gulabjamuns are a delight and so was the combination of manila icecream and chocolate brownie.. please do visit if you can..", 'relative_date': '9 hours ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 16, 402259), 'rating': 5.0, 'username': 'Arclitz', 'n_review_user': '31', 'url_user': 'https://www.google.com/maps/contrib/115106947454017546928/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSURibTRfLWJ3EAE', 'caption': 'Nice behaviour of staff with good food and ambience', 'relative_date': 'a day ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 16, 402259), 'rating': 4.0, 

{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNyMjU2RTRnRRAB', 'caption': None, 'relative_date': 'a month ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 25, 339990), 'rating': 5.0, 'username': 'mohd Zubair Ansari', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/115507967750606940216/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUNyenFydU9REAE', 'caption': "Please don't go to worli...bn...because first they will tell without tax rate ...and after finishing ur food...they will add so much of taxes...and waiting period of...every food serve is..atleast 10 mints...and if ur sit at main entrance table..the staff will forget u...that customer is also sit at that table ..they will not provide u food....properly..so..finally I share my overall experience...please waste ur money...", 'relative_date': 'a month ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 25, 340992), 'rating': 1.0, 'username': 'Maddy S', 'n_review_user': 0, 'url_user': 'https:

{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNMMFlfQ25nRRAB', 'caption': 'Wow super', 'relative_date': 'a month ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 39, 742991), 'rating': 5.0, 'username': 'AMICO ELECTROMAC', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/114703588398037459491/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNMMGJYcHJRRRAB', 'caption': 'Good place to eat', 'relative_date': 'a month ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 39, 743991), 'rating': 5.0, 'username': 'Bijan Padhi', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/112923064036047853618/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUNMMGFYX01BEAE', 'caption': 'Its fantastic staff is well good service is excellent', 'relative_date': 'a month ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 39, 743991), 'rating': 5.0, 'username': 'Mukesh Meena', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/co

{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNMOExYY3NnRRAB', 'caption': 'Worst place, all review are fake', 'relative_date': 'a month ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 49, 679959), 'rating': 1.0, 'username': 'Pankaj Seth', 'n_review_user': '27', 'url_user': 'https://www.google.com/maps/contrib/117588942772471173940/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNMc011UTl3RRAB', 'caption': 'Very nice', 'relative_date': 'a month ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 49, 680959), 'rating': 5.0, 'username': 'sunil kumar', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/107892009926070165632/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSURleW9TRmR3EAE', 'caption': None, 'relative_date': 'a month ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 49, 680959), 'rating': 3.0, 'username': 'sonu k gupta', 'n_review_user': '148', 'url_user': 'https://www.google.com/maps/contrib/106815263517357901035/review

{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSURUaGJUVDRBRRAB', 'caption': None, 'relative_date': '2 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 59, 998237), 'rating': 5.0, 'username': 'Sudha Bar', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/114499676758146039328/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSURUdWZ1Z213RRAB', 'caption': 'This is a great place for dinner, with tasty 😋 food and great atmosphere. ♥️ Thank you for making my special day even more special. 06/05/2024', 'relative_date': '2 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 59, 999237), 'rating': 5.0, 'username': 'Amrisha', 'n_review_user': '139', 'url_user': 'https://www.google.com/maps/contrib/110641127943594479084/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSURUOGMzenBBRRAB', 'caption': 'Best place', 'relative_date': '2 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 17, 55, 59, 999237), 'rating': 5.0, 'username': 'Ravi K

...

{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNEdU83RzlBRRAB', 'caption': 'Really good food very tasty and services also good', 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 4, 6, 121343), 'rating': 5.0, 'username': 'SHELIN RAJ', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/115381786736106168934/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNEdUtyWGxnRRAB', 'caption': None, 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 4, 6, 122340), 'rating': 5.0, 'username': 'Aarif Beg', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/103526450629132555425/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUNEMk92MjFBRRAB', 'caption': None, 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 4, 6, 123339), 'rating': 5.0, 'username': 'Gopi gopi', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/112185606945581062104/reviews?hl

{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUQ5dDUzQVJREAE', 'caption': 'Gud food and taste is priceless', 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 4, 29, 880259), 'rating': 5.0, 'username': 'Akshay Akki', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/118091024736368471546/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUQ5dF9YVnl3RRAB', 'caption': None, 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 4, 29, 880259), 'rating': 5.0, 'username': 'Kishore K', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/104429472969128125716/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUQ5dC1HdEx3EAE', 'caption': 'If you are with your friends and family I would suggest you this amazing place with great atmosphere and delicious food 😋 …', 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 4, 29, 881259), 'rating': 5.0, 'username': 'Aditya Si

{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUQ5X2NpZ1NnEAE', 'caption': 'Good food', 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 4, 58, 26333), 'rating': 5.0, 'username': 'prakash pandey', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/109112324065959434224/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUQ5dmJ2TVhBEAE', 'caption': 'Really such a Good experience. I suggested its good for your family enjoyment. Please try......', 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 4, 58, 26333), 'rating': 5.0, 'username': 'Nootan kuntal', 'n_review_user': '3', 'url_user': 'https://www.google.com/maps/contrib/112627210704721810740/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUQ5dmQyTDBRRRAB', 'caption': None, 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 4, 58, 27546), 'rating': 1.0, 'username': 'Dileep Mudhiraj', 'n_review_user': 0, 'url_user': 'ht

{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUQ5bWFTNy1BRRAB', 'caption': 'Very nice atmosphere and sorv system very well, food is tasty', 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 5, 24, 180058), 'rating': 5.0, 'username': 'नशा मुक्ति टिप्स द विश्नोई', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/106694622820344405167/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUQ5cWFfMjV3RRAB', 'caption': 'Hotal staff is sincere give good service. Food quality and tast good', 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 5, 24, 180058), 'rating': 5.0, 'username': 'ASHISH TAMGADGE', 'n_review_user': '10', 'url_user': 'https://www.google.com/maps/contrib/110593823471328039196/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUQ5cWR1LWJnEAE', 'caption': 'Good', 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 5, 24, 180058), 'rating': 5.0, 'username': 'Narra S

{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUQ5NXV1azZ3RRAB', 'caption': 'Good service..food is also very tasty...had a great time here..must try', 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 5, 51, 278711), 'rating': 5.0, 'username': 'Savita Chauhan', 'n_review_user': 0, 'url_user': 'https://www.google.com/maps/contrib/108370234690564346269/reviews?hl=en'}
{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUQ5NXNXbWFBEAE', 'caption': 'Fake and very dirty hotel 😡😡😡🤬🤬🤬🤬🤬 …', 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 5, 51, 278711), 'rating': 1.0, 'username': 'Lucky Saini', 'n_review_user': '6', 'url_user': 'https://www.google.com/maps/contrib/103802104020560752674/reviews?hl=en'}
{'id_review': 'ChdDSUhNMG9nS0VJQ0FnSUQ5NXVTOW1RRRAB', 'caption': None, 'relative_date': '4 months ago', 'retrieval_date': datetime.datetime(2024, 8, 11, 18, 5, 51, 278711), 'rating': 5.0, 'username': 'N K Jagadish', 'n_review_user': '27', 'url_user'